In [2]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

In [4]:
# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [5]:
intents

{u'intents': [{u'context_set': u'',
   u'patterns': [u'Hi',
    u'How are you',
    u'Is anyone there?',
    u'Hello',
    u'Good day'],
   u'responses': [u'Hello, thanks for visiting',
    u'Good to see you again',
    u'Hi there, how can I help?'],
   u'tag': u'greeting'},
  {u'patterns': [u'Bye', u'See you later', u'Goodbye'],
   u'responses': [u'See you later, thanks for visiting',
    u'Have a nice day',
    u'Bye! Come back again soon.'],
   u'tag': u'goodbye'},
  {u'patterns': [u'Thanks', u'Thank you', u"That's helpful"],
   u'responses': [u'Happy to help!', u'Any time!', u'My pleasure'],
   u'tag': u'thanks'},
  {u'patterns': [u'What hours are you open?',
    u'What are your hours?',
    u'When are you open?'],
   u'responses': [u"We're open every day 9am-9pm",
    u'Our hours are 9am-9pm every day'],
   u'tag': u'hours'},
  {u'patterns': [u'Which mopeds do you have?',
    u'What kinds of mopeds are there?',
    u'What do you rent?'],
   u'responses': [u'We rent Yamaha, Piaggio

In [6]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

(27, 'documents')
(9, 'classes', [u'goodbye', u'greeting', u'hours', u'mopeds', u'opentoday', u'payments', u'rental', u'thanks', u'today'])
(48, 'unique stemmed words', [u"'d", u"'s", u'a', u'acceiv', u'anyon', u'ar', u'bye', u'can', u'card', u'cash', u'credit', u'day', u'do', u'doe', u'good', u'goodby', u'hav', u'hello', u'help', u'hi', u'hour', u'how', u'i', u'is', u'kind', u'lat', u'lik', u'mastercard', u'mop', u'of', u'on', u'op', u'rent', u'see', u'tak', u'thank', u'that', u'ther', u'thi', u'to', u'today', u'we', u'what', u'when', u'which', u'work', u'yo', u'you'])


In [7]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

In [8]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 3999  | total loss: 0.14870 | time: 0.013s
| Adam | epoch: 1000 | loss: 0.14870 - acc: 0.9919 -- iter: 24/27
Training Step: 4000  | total loss: 0.15005 | time: 0.016s
| Adam | epoch: 1000 | loss: 0.15005 - acc: 0.9927 -- iter: 27/27
--
INFO:tensorflow:/media/david13/OS/1_ai_dev/2_Chatbot_development/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


INFO:tensorflow:/media/david13/OS/1_ai_dev/2_Chatbot_development/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [9]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [10]:
p = bow("is your shop open today?", words)
print (p)
print (classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 1 0]
[u'goodbye', u'greeting', u'hours', u'mopeds', u'opentoday', u'payments', u'rental', u'thanks', u'today']


In [11]:
print(model.predict([p]))

[[6.6807913e-04 2.3155130e-06 3.5540075e-03 2.7501621e-06 7.2586501e-01
  1.2480945e-02 2.2843989e-07 2.4994335e-03 2.5492725e-01]]


In [12]:
# save all of our data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )